# **Importing and preprocessing data**

In [2]:
import os
import csv
!pip install pyspark
!pip install findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder.appName("AmazonBabyReview").getOrCreate()
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql.functions import explode
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
from pyspark.sql.functions import lit, collect_set, col, expr, collect_list, countDistinct, count, sum



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=655b7daa5087553d431bb2389db4ca28c5b2f414eca7013c813ea33113973a2b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
os.environ['KAGGLE_USERNAME'] = "hoadao12"
os.environ['KAGGLE_KEY'] = "2c070edb8c2c2b2c75c72de73ed5c781"
!kaggle datasets download -d cynthiarempel/amazon-us-customer-reviews-dataset

100% 20.9G/21.0G [03:17<00:00, 32.6MB/s]
100% 21.0G/21.0G [03:18<00:00, 114MB/s] 


In [4]:
import pandas as pd
import zipfile
import numpy as np

with zipfile.ZipFile('/content/amazon-us-customer-reviews-dataset.zip', 'r') as zip_ref:
    with zip_ref.open('amazon_reviews_us_Baby_v1_00.tsv') as file:
        selected_columns = ['customer_id', 'review_id', 'product_id', 'product_title']
        df = pd.read_csv(file, delimiter='\t', usecols=selected_columns)
df.head()
num_rows = df.shape[0]
print("Total rows:",num_rows)

Total rows: 1749190


In [5]:
total_init_rows = df.shape[0]
df = df.drop_duplicates()
no_duplicates_rows = df.shape[0]
customerid_counts = df['customer_id'].value_counts()
valid_customerid = customerid_counts[customerid_counts >= 2].index.tolist()
newdf = df[df['customer_id'].isin(valid_customerid)]
rows = newdf.shape[0]
print("Total new rows:",rows)
print("Dropped the", round(1-rows/total_init_rows, 4)*100, "% of the inital rows (", total_init_rows,")")
print(" - the", round(1-no_duplicates_rows/total_init_rows, 4)*100, "% are duplicates")
print(" - the", round((no_duplicates_rows - rows)/total_init_rows, 4)*100, "% of customers with one review")
newdf.head()

Total new rows: 1070716
Dropped the 38.79 % of the inital rows ( 1749190 )
 - the 0.0 % are duplicates
 - the 38.79 % of customers with one review


,customer_id,review_id,product_id,product_title
0,9970739,R8EWA1OFT84NX,B00GSP5D94,Summer Infant SwaddleMe Adjustable Infant Wrap...
1,23538442,R2JWY4YRQD4FOP,B00YYDDZGU,Pacifier Clip Girl (3 Pack) Ziggy Baby 2-Sided...
2,8273344,RL5ESX231LZ0B,B00BUBNZC8,Udder Covers - Breast Feeding Nursing Cover
3,24557753,RRMS9ZWJ2KD08,B00AWLZFTS,Gerber Graduates Fun Pack Utensils
4,46263340,R14I3ZG5E6S7YM,B00KM60D3Q,Summer Infant Ultra Sight Pan/Scan/Zoom Video ...


In [6]:
sdf = spark.createDataFrame(newdf)
sdf.show(10)

+-----------+--------------+----------+--------------------+
|customer_id|     review_id|product_id|       product_title|
+-----------+--------------+----------+--------------------+
|    9970739| R8EWA1OFT84NX|B00GSP5D94|Summer Infant Swa...|
|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|Pacifier Clip Gir...|
|    8273344| RL5ESX231LZ0B|B00BUBNZC8|Udder Covers - Br...|
|   24557753| RRMS9ZWJ2KD08|B00AWLZFTS|Gerber Graduates ...|
|   46263340|R14I3ZG5E6S7YM|B00KM60D3Q|Summer Infant Ult...|
|   24557753|R13EPSFP5DODN5|B00PQMRZG4|Summer Infant Kee...|
|   33520065| R6RBP4HTE67SY|B005DL5970|Natural HE Powder...|
|   20241560|R15B3EU40RSU2W|B00C6D2WL4|Dr. Brown's Bottl...|
|   52570308|R2C99DJEO4RZ4K|B00RLYG2S2|Abiie Beyond Wood...|
|    9287389| REV51EW323H8W|B010UX9T5I|Lovinglove Baby G...|
+-----------+--------------+----------+--------------------+
only showing top 10 rows



## **Link Analysis**



1.   Construct links
2.   Transitional Probabilities
3.   Calculation



In [7]:
# Group the product IDs by customer ID
grouped_df = sdf.groupBy("customer_id").agg(F.collect_list("product_id").alias("products"))
grouped_df.show(truncate=False)


+-----------+------------------------------------------------------------------------------------------------+
|customer_id|products                                                                                        |
+-----------+------------------------------------------------------------------------------------------------+
|10227      |[B001MUVX8U, B001DDMFK6, B000YZ6ZHS, B001KZH69M]                                                |
|11170      |[B00D43C134, B00E4HUFTG, B00E1CI2MG]                                                            |
|11316      |[B004GIGUW4, B004GIQB28]                                                                        |
|11937      |[B00INK71U6, B001MS82HM]                                                                        |
|11942      |[B00KCNJAMY, B00KCNJ1BY, B00K92MWE6]                                                            |
|12280      |[B000FI1YKW, B005IWM7V0, B00H8MRBKA]                                                            |
|

In [8]:
# Explode the list of products
exploded_df = grouped_df.select("customer_id", explode("products").alias("product_id"))
links_df = exploded_df.alias("a").join(exploded_df.alias("b"), "customer_id").filter("a.product_id <> b.product_id").selectExpr(
    "a.product_id as product_a","b.product_id as product_b")


In [9]:
links_df.show(10)

+----------+----------+
| product_a| product_b|
+----------+----------+
|B001MUVX8U|B001DDMFK6|
|B001MUVX8U|B000YZ6ZHS|
|B001MUVX8U|B001KZH69M|
|B001DDMFK6|B001MUVX8U|
|B001DDMFK6|B000YZ6ZHS|
|B001DDMFK6|B001KZH69M|
|B000YZ6ZHS|B001MUVX8U|
|B000YZ6ZHS|B001DDMFK6|
|B000YZ6ZHS|B001KZH69M|
|B001KZH69M|B001MUVX8U|
+----------+----------+
only showing top 10 rows



In [10]:
# Count links of each 'product_a'
count_product_a= links_df.groupBy('product_a').agg(count("*").alias("links_product_a"))
# Calculate the probability of being at each product a to next being at each of the other pages
transitional_prob = links_df.join(count_product_a, 'product_a').withColumn("probability", 1 / col('links_product_a'))
transitional_prob.show(20)

+----------+----------+---------------+--------------------+
| product_a| product_b|links_product_a|         probability|
+----------+----------+---------------+--------------------+
|B0006HBS1M|B004EWGDCE|           6653|1.503081316699233...|
|B007BEHSDU|B0034XQXB0|           1769|5.652911249293386E-4|
|B007BEHSDU|B00HF3Y100|           1769|5.652911249293386E-4|
|B007BEHSDU|B003LNU8PK|           1769|5.652911249293386E-4|
|B011J5TUUU|B0126D8J9M|             28| 0.03571428571428571|
|B008BRVTLI|B00R9QNCQW|             15| 0.06666666666666667|
|B00LFB616A|B00Q29MYN4|             27|0.037037037037037035|
|B00LFB616A|B00Q29MYOS|             27|0.037037037037037035|
|B004YL3332|B00171WXII|            735|0.001360544217687...|
|B004YL3332|B0071D1AKI|            735|0.001360544217687...|
|B004YL3332|B0034G60JM|            735|0.001360544217687...|
|B00078ZHPS|B00EPFLHVM|           1185|8.438818565400844E-4|
|B00078ZHPS|B0038JDUBQ|           1185|8.438818565400844E-4|
|B00078ZHPS|B00IX6PEB8| 

In [11]:
#check sum transitional probabilities of random product
check_sum = transitional_prob.filter(col("product_a") == "B00G34TL2U").agg({"probability": "sum"}).collect()[0][0]
print(check_sum)

1.000000000000003


In [12]:
# Calculate the distinct a
n = transitional_prob.select("product_a").distinct().count()

# Initial Pagerank
initial = [(row["product_a"], 1 / n) for row in transitional_prob.select("product_a").distinct().collect()]
page_rank = spark.createDataFrame(initial, ["product", "page_rank"])
page_rank.show()

+----------+--------------------+
|   product|           page_rank|
+----------+--------------------+
|B0006HBS1M|8.250620859219656E-6|
|B007BEHSDU|8.250620859219656E-6|
|B011J5TUUU|8.250620859219656E-6|
|B008BRVTLI|8.250620859219656E-6|
|B00LFB616A|8.250620859219656E-6|
|B004YL3332|8.250620859219656E-6|
|B00078ZHPS|8.250620859219656E-6|
|B00QZ3UNIU|8.250620859219656E-6|
|B00U0DFCGO|8.250620859219656E-6|
|B00E3RKC36|8.250620859219656E-6|
|B00PF84ANA|8.250620859219656E-6|
|B00JPSU8W8|8.250620859219656E-6|
|B00MUVQS12|8.250620859219656E-6|
|B00YTXSTES|8.250620859219656E-6|
|B00O3GVSZI|8.250620859219656E-6|
|B00BEVSRU4|8.250620859219656E-6|
|B00PF84ATO|8.250620859219656E-6|
|B00MAGCCLC|8.250620859219656E-6|
|B00L8JIO0K|8.250620859219656E-6|
|B006STJTR2|8.250620859219656E-6|
+----------+--------------------+
only showing top 20 rows



In [13]:
iteration = 0
iterations = 20
tolerance = 5e-9
distance = float('inf')

while iteration < iterations and distance >= tolerance:
    new_pagerank = (transitional_prob.join(page_rank, transitional_prob.product_a == page_rank.product, "inner")
                      .select("product_b", (F.col("page_rank") * F.col("probability")).alias("new_pagerank"))
                      .groupBy("product_b")
                      .agg(F.sum("new_pagerank").alias("new_pagerank")))

    #Squared Euclidean distance
    distance = (new_pagerank
              .join(page_rank, new_pagerank.product_b == page_rank.product)
              .withColumn('distance', F.abs(F.col("new_pagerank") - col("page_rank")))
              .agg(F.sum(F.pow('distance', 2)).alias('total_distance'))
              .head()['total_distance'])

    print(f"Iteration: {iteration+1}, Total_distance: {distance}")

    page_rank = new_pagerank.withColumnRenamed('new_pagerank', 'page_rank').withColumnRenamed('product_b', 'product')

    iteration += 1


Iteration: 1, Total_distance: 9.36949534295732e-05
Iteration: 2, Total_distance: 7.154753537913271e-06
Iteration: 3, Total_distance: 7.041879013819292e-07
Iteration: 4, Total_distance: 2.0250065799734934e-07
Iteration: 5, Total_distance: 6.109819985532666e-08
Iteration: 6, Total_distance: 2.552813834008041e-08
Iteration: 7, Total_distance: 1.232612187744269e-08
Iteration: 8, Total_distance: 7.647663478632765e-09
Iteration: 9, Total_distance: 5.394293113870115e-09
Iteration: 10, Total_distance: 4.500797487443926e-09


In [15]:
page_rank.show()

+----------+--------------------+
|   product|           page_rank|
+----------+--------------------+
|B00699FWD6|2.331197611075329E-6|
|B003TJ9PDC|1.753911420481256E-5|
|B00JLI73ZM|3.807141442074022...|
|B00O3GVSZI|5.114570327617026E-5|
|B0013HI718|3.424252421109331E-6|
|B001QXCF1C|1.633640633632889...|
|B00AHVR3ES|2.616634045486809...|
|B00E9RMHD8|1.715410809635548...|
|B00G6R6KVO|6.891847679417847E-7|
|B0006HBS1M|9.138603552230531E-4|
|B004V23YJM|1.137411902645144...|
|B00U0DD6EO|2.063240206200156...|
|B007BEHSDU|2.424804641068821...|
|B00G3XR8PS|9.568021643431567E-5|
|B00FP1XKVK|5.756984433978033...|
|B004GCJML6|1.011628999485941...|
|B0035EQKF2|2.843061383011291E-6|
|B004EL0G9Q|7.548233381351409E-6|
|B00F7PXANW|3.881080727479839E-5|
|B003HAI7CI|4.128463486203096...|
+----------+--------------------+
only showing top 20 rows



In [17]:
#Join pagerank df with the original df
final_df = page_rank.join(sdf, page_rank.product == sdf.product_id).select("product", "page_rank", "product_title").distinct()

# Show some products with details
highest_prod = final_df.orderBy(F.desc("page_rank")).limit(8)
print("Top products:")
highest_prod.show(truncate=False)
least_prod = final_df.orderBy(F.asc("page_rank")).limit(8)
print("Least products:")
least_prod.show(truncate=False)


Top products:
+----------+---------------------+-----------------------------------------------------+
|product   |page_rank            |product_title                                        |
+----------+---------------------+-----------------------------------------------------+
|B000YDDF6O|0.002834902848013501 |Baby Einstein Take Along Tunes Musical Toy           |
|B000IDSLOG|0.002743799720272408 |Vulli Sophie la Girafe                               |
|B00171WXII|0.0027176925737394217|FridaBaby NoseFrida The SnotSucker Nasal Aspirator   |
|B002QYW8LW|0.0019224379509214986|Baby Banana Toothbrush                               |
|B009EDSWJA|0.001425089010029274 |Summer Infant 5-Piece Essentials Diaper Changing Kits|
|B0052QYLUM|0.0013910880737661613|Infant Optics DXR-5 Portable Video Baby Monitor      |
|B001WAJVZM|0.001328465103660608 |Skip Hop Moby Bath Spout Cover Universal Fit, Blue   |
|B00295MQLU|0.001218435568982723 |Simple Wishes Hands Free Breastpump Bra              |
+------